### 📦 Data ETL Pipeline – Brazilian E-commerce Dataset

This notebook demonstrates a complete **ETL (Extract, Transform, Load)** pipeline for the Brazilian E-commerce Public Dataset. The process begins with **fetching the dataset from Kaggle**, followed by a comprehensive **data cleaning and transformation workflow**. The final cleaned datasets are then loaded into a **PostgreSQL database** for downstream analysis and reporting.

Key operations include:

* Dataset extraction using Kaggle API
* Data type correction and normalization
* Handling of missing, duplicate, and inconsistent values
* Text translation and label standardization
* Integration and validation across related tables
* Data loading into a relational database

This pipeline ensures the dataset is clean, consistent, and ready for robust analytical processing.          
* Dataset Link:  https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

# 1. Extract

In [ ]:
import os
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi

# Set up Kaggle API
api = KaggleApi()
api.authenticate()

# Define the dataset and target directory
dataset = 'olistbr/brazilian-ecommerce'
download_dir = 'data'
zip_file_name = 'brazilian-ecommerce.zip'  # Corrected file name
zip_file_path = os.path.join(download_dir, zip_file_name)

# Create the data folder if it doesn't exist
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# Download the dataset if not already present
if not os.path.exists(zip_file_path):
    print("Downloading dataset...")
    api.dataset_download_files(dataset, path=download_dir, unzip=False)
else:
    print("Dataset already downloaded.")

# Extract the dataset
if os.path.exists(zip_file_path):
    print("Extracting dataset...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(download_dir)
    print(f"Dataset extracted to {download_dir}")
else:
    print(f"Error: ZIP file not found at {zip_file_path}")

# 2. Transform 

## 2.1. Importing Libraries

In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

## 2.2. Loading Data to the dataframe

In [5]:
# Reading data
orders = pd.read_csv("data/olist_orders_dataset.csv")                
sellers = pd.read_csv("data/olist_sellers_dataset.csv")                            
items = pd.read_csv("data/olist_order_items_dataset.csv")          
category = pd.read_csv("data/product_category_name_translation.csv")              
payments = pd.read_csv("data/olist_order_payments_dataset.csv")
customers = pd.read_csv("data/olist_customers_dataset.csv")             
reviews = pd.read_csv("data/olist_order_reviews_dataset.csv")
geolocation = pd.read_csv("data/olist_geolocation_dataset.csv")           
products = pd.read_csv("data/olist_products_dataset.csv")

## 2.3. Extracting Basic Information about the dataframes

In [7]:
# Extracting Basic Information of our data
print('Order Dataset :', orders.shape)
print('Sellers Dataset :', sellers.shape)
print('Items Dataset :', items.shape)
print('Products Dataset :', products.shape)
print('Payments Dataset :', payments.shape)
print('Customers Dataset :', customers.shape)
print('Reviews Dataset :', reviews.shape)
print('Geolocation Dataset :', geolocation.shape)

Order Dataset : (99441, 8)
Sellers Dataset : (3095, 4)
Items Dataset : (112650, 7)
Products Dataset : (32951, 9)
Payments Dataset : (103886, 5)
Customers Dataset : (99441, 5)
Reviews Dataset : (99224, 7)
Geolocation Dataset : (1000163, 5)


## 2.4. Preview of each dataframe

In [9]:
# Fetch first few records from Order Dataset
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [11]:
# Fetch first few records from Seller Dataset
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [13]:
# Fetch first few records from Items Dataset
items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [15]:
# Fetch first few records from Producta Dataset
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [17]:
# Fetch first few records from Payment Dataset
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [19]:
# Fetch first few records from Customers Dataset
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [21]:
# Fetch first few records from Reviews Dataset
reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [23]:
# Fetch first few records from Geolocation Dataset
geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


## 2.5. Information of each dataframe

In [26]:
# Basic Information 
# Dictionary of all your DataFrames
dataframes = {
    "Orders": orders,
    "Sellers": sellers,
    "Items": items,
    "Products": products,
    "Payments": payments,
    "Customers": customers,
    "Reviews": reviews,
    "Geolocation": geolocation
}

# Print .info() with spacing
for name, df in dataframes.items():
    print(f"========== {name} ==========")
    df.info()
    print("\n\n") 

========== Orders ==========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB



========== Sellers ==========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  -------------

## 2.6. Missing Value Checking

In [28]:
# Dictionary of all your DataFrames
dataframes = {
    "Orders": orders,
    "Sellers": sellers,
    "Items": items,
    "Products": products,
    "Payments": payments,
    "Customers": customers,
    "Reviews": reviews,
    "Geolocation": geolocation
}

In [30]:
# Check for nulls
for name, df in dataframes.items():
    print(f"========== Missing Values in {name} ==========")
    print(df.isnull().sum())
    print("\n") 

========== Missing Values in Orders ==========
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64


========== Missing Values in Sellers ==========
seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64


========== Missing Values in Items ==========
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64


========== Missing Values in Products ==========
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
prod

## 2.7. Duplicate Value Checking

In [32]:
# Dictionary of all your DataFrames
dataframes = {
    "Orders": orders,
    "Sellers": sellers,
    "Items": items,
    "Products": products,
    "Payments": payments,
    "Customers": customers,
    "Reviews": reviews,
    "Geolocation": geolocation
}

In [34]:
# Check for duplicates
for name, df in dataframes.items():
    print(f"========== Duplicate Rows in {name} ==========")
    print(f"Duplicate count: {df.duplicated().sum()}")
    print("\n") 

========== Duplicate Rows in Orders ==========
Duplicate count: 0


========== Duplicate Rows in Sellers ==========
Duplicate count: 0


========== Duplicate Rows in Items ==========
Duplicate count: 0


========== Duplicate Rows in Products ==========
Duplicate count: 0


========== Duplicate Rows in Payments ==========
Duplicate count: 0


========== Duplicate Rows in Customers ==========
Duplicate count: 0


========== Duplicate Rows in Reviews ==========
Duplicate count: 0


========== Duplicate Rows in Geolocation ==========
Duplicate count: 261831




## 2.8. Outliers Checking

In [36]:
# Statistical Summary of Order Dataframe
orders.describe()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
count,99441,99441,99441,99441,99281,97658,96476,99441
unique,99441,99441,8,98875,90733,81018,95664,459
top,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2018-04-11 10:48:14,2018-02-27 04:31:10,2018-05-09 15:48:00,2018-05-08 23:38:46,2017-12-20 00:00:00
freq,1,1,96478,3,9,47,3,522


In [38]:
# Statistical Summary of Sellers Dataframe
sellers.describe()

,seller_zip_code_prefix
count,3095.000000
mean,32291.059451
std,32713.453830
min,1001.000000
25%,7093.500000
50%,14940.000000
75%,64552.500000
max,99730.000000


In [40]:
# Statistical Summary of Items Dataframe
items.describe()

,order_item_id,price,freight_value
count,112650.000000,112650.000000,112650.000000
mean,1.197834,120.653739,19.990320
std,0.705124,183.633928,15.806405
min,1.000000,0.850000,0.000000
25%,1.000000,39.900000,13.080000
50%,1.000000,74.990000,16.260000
75%,1.000000,134.900000,21.150000
max,21.000000,6735.000000,409.680000


In [42]:
# Statistical Summary of Products Dataframe
products.describe()

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32341.000000,32341.000000,32341.000000,32949.000000,32949.000000,32949.000000,32949.000000
mean,48.476949,771.495285,2.188986,2276.472488,30.815078,16.937661,23.196728
std,10.245741,635.115225,1.736766,4282.038731,16.914458,13.637554,12.079047
min,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,42.000000,339.000000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,51.000000,595.000000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,57.000000,972.000000,3.000000,1900.000000,38.000000,21.000000,30.000000
max,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000


In [44]:
# Statistical Summary of Paymengt Dataframe
payments.describe()

,payment_sequential,payment_installments,payment_value
count,103886.000000,103886.000000,103886.000000
mean,1.092679,2.853349,154.100380
std,0.706584,2.687051,217.494064
min,1.000000,0.000000,0.000000
25%,1.000000,1.000000,56.790000
50%,1.000000,1.000000,100.000000
75%,1.000000,4.000000,171.837500
max,29.000000,24.000000,13664.080000


In [46]:
# Statistical Summary of Customers Dataframe
customers.describe()

,customer_zip_code_prefix
count,99441.000000
mean,35137.474583
std,29797.938996
min,1003.000000
25%,11347.000000
50%,24416.000000
75%,58900.000000
max,99990.000000


In [48]:
# Statistical Summary OF Reviews Dataframe
reviews.describe()

,review_score
count,99224.000000
mean,4.086421
std,1.347579
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [50]:
# Statistical Summary of Geolocation Dataframe
geolocation.describe()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
count,1.000163e+06,1.000163e+06,1.000163e+06
mean,3.657417e+04,-2.117615e+01,-4.639054e+01
std,3.054934e+04,5.715866e+00,4.269748e+00
min,1.001000e+03,-3.660537e+01,-1.014668e+02
25%,1.107500e+04,-2.360355e+01,-4.857317e+01
50%,2.653000e+04,-2.291938e+01,-4.663788e+01
75%,6.350400e+04,-1.997962e+01,-4.376771e+01
max,9.999000e+04,4.506593e+01,1.211054e+02


In [54]:
import pandas as pd

def detect_outliers_iqr(df, df_name):
    print(f"\n📊 Outlier Summary for: {df_name}")
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower) | (df[col] > upper)]
        print(f"🔹 {col}: {len(outliers)} outliers")

# Apply to each relevant DataFrame
detect_outliers_iqr(orders, 'Orders')
detect_outliers_iqr(items, 'Order Items')
detect_outliers_iqr(products, 'Products')
detect_outliers_iqr(payments, 'Payments')
detect_outliers_iqr(customers, 'Customers')
detect_outliers_iqr(sellers, 'Sellers')
detect_outliers_iqr(reviews, 'Reviews')


📊 Outlier Summary for: Orders

📊 Outlier Summary for: Order Items
🔹 order_item_id: 13984 outliers
🔹 price: 8427 outliers
🔹 freight_value: 12134 outliers

📊 Outlier Summary for: Products
🔹 product_name_lenght: 290 outliers
🔹 product_description_lenght: 2078 outliers
🔹 product_photos_qty: 849 outliers
🔹 product_weight_g: 4551 outliers
🔹 product_length_cm: 1380 outliers
🔹 product_height_cm: 1892 outliers
🔹 product_width_cm: 912 outliers

📊 Outlier Summary for: Payments
🔹 payment_sequential: 4526 outliers
🔹 payment_installments: 6313 outliers
🔹 payment_value: 7981 outliers

📊 Outlier Summary for: Customers
🔹 customer_zip_code_prefix: 0 outliers

📊 Outlier Summary for: Sellers
🔹 seller_zip_code_prefix: 0 outliers

📊 Outlier Summary for: Reviews
🔹 review_score: 14575 outliers


## Table of Assessment Summary

| **NO** |    **TABLE NAME**     |                                                                                       **DATA TYPE**                                                                                       |                                                                                                 **MISSING VALUE**                                                                                                 |                             **DUPLICATE DATA**                             |                **INACCURATE VALUE**                 |
| :----: | :-------------------: | :---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------: | :---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------: | :------------------------------------------------------------------------: | :-------------------------------------------------: |
|   1.   |       customers       |                                                             There is one invalid data type at column customer_zip_code_prefix                                                             |                                                                                                         -                                                                                                         |                                     -                                      |                          -                          |
|   2.   |        orders         |                                         There are five invalid data type starts from order_purchase_timestamp until order_estimated_delivery_date                                         |                                            There are three columns with missing values: order_approved_at, order_delivered_carrier_date, order_delivered_customer_date                                            |                                     -                                      |                          -                          |
|   3.   |      items      |                                                                  There is one invalid data type in column order_item_id                                                                   |                                                                                                         -                                                                                                         | Tranform the order_item_id into qty to extract unit-per-order line profile | There is outliers in column price and freight_value |
|   4.   |    payments     |                                                                                             -                                                                                             |                                                                                                         -                                                                                                         |                                     -                                      |     There is an outlier in column payment_value     |
|   5.   |     reviews     |                                                There are two invalid data type at columns review_creation_date and review_answer_timestamp                                                |                                                              There are two columns with missing values: review_comment_title review_comment_message                                                               |                                     -                                      |                          -                          |
|   6.   |       products        | There are seven columns needs to change, product_name_length, product_description_length, product_photos_qty, product_weight_g, product_length_cm, product_height_cm and product_width_cm | There are eight columns with missing values: product_category_name, product_name_lenght, product_description_lenght, product_photos_qty, product_weight_g, product_length_cm, product_height_cm, product_width_cm |                                     -                                      |                          -                          |
|   7.   | product_category_name |                                                                                             -                                                                                             |                                                                                                         -                                                                                                         |                                     -                                      |                          -                          |
|   8.   |        sellers        |                                                              There is one invalid data type at column seller_zip_code_prefix                                                              |                                                                                                         -                                                                                                         |                                     -                                      |                          -                          |
|   9.   |      geolocation      |                                                           There is one invalid data type at column geolocation_zip_code_prefix                                                            |                                                                                                         -                                                                                                         |                       Total duplicate values: 261831                       |                          -                          |


## Fixing Data Types 

In [57]:
# Handling Datatypes
# Convert to datetime in Orders
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])

# Convert to datetime in Items
items['shipping_limit_date'] = pd.to_datetime(items['shipping_limit_date'])

# Convert to datetime in Reviews
reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])
reviews['review_answer_timestamp'] = pd.to_datetime(reviews['review_answer_timestamp'])

## Translate & Normalize Labels   

In [59]:
# Joining Product with Product Categoty
products = products.merge(category, how='left', on='product_category_name')
products['product_category_name'] = products['product_category_name_english']
products.drop('product_category_name_english', axis=1, inplace=True)

In [ ]:
products.product_category_name.isna().sum()

In [121]:
import unicodedata

def normalize_proper_case(text):
    if isinstance(text, str):
        # Normalize unicode (remove accents)
        no_accent = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
        # Convert to proper/title case
        return no_accent.title()
    return text

# Apply to geolocation_city
geolocation['geolocation_city'] = geolocation['geolocation_city'].apply(normalize_proper_case)

## Handle Missing Values  

In [61]:
# Order Dataframe 
# Fill missing delivery dates temporarily (⚠️ use with caution)
orders['order_delivered_customer_date'] = orders['order_delivered_customer_date'].ffill()

# Check missing values after ffill
print("Missing values:", orders["order_delivered_customer_date"].isnull().sum())

Missing values: 0


## Handling Duplicates  

In [125]:
# Only geolocation dataset has duplicate records, so we neeed to drop them. 
geolocation.duplicated().sum()  

279668

In [123]:
# Only geolocation dataset has duplicate records, so we neeed to drop them. 
geolocation.duplicated().sum()  

279668

In [95]:
filtered_duplicates = duplicates[
    (duplicates.geolocation_zip_code_prefix == 1046) &
    (duplicates.geolocation_lat.between(-23.546082, -23.546080)) &
    (duplicates.geolocation_lng.between(-46.644821, -46.644819))
]

filtered_duplicates

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
15,1046,-23.546081,-46.64482,sao paulo,SP
44,1046,-23.546081,-46.64482,sao paulo,SP
65,1046,-23.546081,-46.64482,sao paulo,SP
67,1046,-23.546081,-46.64482,sao paulo,SP
82,1046,-23.546081,-46.64482,sao paulo,SP
94,1046,-23.546081,-46.64482,sao paulo,SP
220,1046,-23.546081,-46.64482,sao paulo,SP
255,1046,-23.546081,-46.64482,sao paulo,SP
343,1046,-23.546081,-46.64482,sao paulo,SP
361,1046,-23.546081,-46.64482,sao paulo,SP


## Fix Inconsistent Values  

## Handle Outliers & Redundancy  

## Normalize Numerical Data (if modeling)

1. Orders Table
* Difference between estimated delivery date and true delivery date (shipping_time_delta)
* Difference between order date and delivery date (shipping_duration)
* Difference between order date and estimated delivery date (estimated_duration)

In [ ]:
# Add ordinal date
orders["date_ordinal"] = orders["order_purchase_timestamp"].apply(lambda date: date.toordinal())

# Feature engineering on durations
orders["shipping_time"] = orders["order_estimated_delivery_date"] - orders["order_delivered_customer_date"]
orders["shipping_duration"] = orders["order_delivered_customer_date"] - orders["order_purchase_timestamp"]
orders["estimated_duration"] = orders["order_estimated_delivery_date"] - orders["order_purchase_timestamp"]

# 3. Load (to PostgreSQL)